In [1]:
import pandas as pd

In [2]:
# read in file and get sheet names
file = '../Tissue-Specific Cit IDs.xlsx'
xl = pd.ExcelFile(file)
sheets = xl.sheet_names
sheets = sheets[1:] # 'Summary page not needed'
brain = sheets[:5]
organs = sheets[5:]

# create master dataframe of all proteins in all tissue
proteins = pd.DataFrame()
for sheet in sheets:
    df = pd.read_excel(file, sheet_name=sheet, usecols=range(0, 1)).dropna()
    df.columns = [sheet]
    if proteins.empty:
        proteins = df
    else:
        proteins = proteins.join(df, how='outer')

In [3]:
# create new dataframe with count of shared proteins
# in protein sections
protein_share = pd.DataFrame()
for source in proteins:
    found = proteins[source]
    found_df = pd.DataFrame(found)
    for target in proteins:
        portion = {}
        if source == target:
            other = pd.melt(proteins[[c for c in proteins if c != source]])
            other = other.dropna()
            sub = found_df[~found_df[source].isin(other.value)].dropna()
        else:
            sub = proteins[proteins[target].isin(found)][target]
            sub = sub.dropna()
        portion['source_tissue'] = source
        portion['comp_tissue'] = target
        portion['overlapping'] = len(sub)
        portion['value_norm'] = len(sub) / len(found)
        if protein_share.empty:
            protein_share = pd.DataFrame(portion,
                    index=range(1))
        else:
            protein_share = pd.concat([protein_share,
                    pd.DataFrame(portion, index=range(1))])

In [4]:
regions = [r for r in protein_share.source_tissue.unique()]
vals = []
fin_text = []
fin_text.append(f'''Creator "igraph version 0.6 Wed Jan 30 10:28:57 2013"
Version 1
graph
[
  directed 0''')
for r in regions:
    vals.append(regions.index(r))
    text = f'''
  node
  [
    id {regions.index(r)}
    label "{r}"
    group {regions.index(r)}
    fill "#BEBADA"
    border "#aba7c4"
  ]'''
    fin_text.append(text)
t_map = dict(zip(regions, vals))
protein_share.loc[:, 'source_tissue'] = protein_share.source_tissue.map(t_map)
protein_share.loc[:, 'comp_tissue'] = protein_share.comp_tissue.map(t_map)

for r in protein_share.itertuples():
  text = f'''
  edge
  [
    source {r.source_tissue}
    target {r.comp_tissue}
    value {r.overlapping} 
  ]'''
  fin_text.append(text)
fin_text.append(']')

In [5]:
with open('protein.gml.txt', 'w') as f:
    f.write(''.join(fin_text))